<a href="https://colab.research.google.com/github/nicikess/hsg-ml-course/blob/main/morning_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import pandas as pd
import numpy as np
from PIL import Image

!pip install rasterio
!pip install tqdm
import rasterio
from rasterio.plot import reshape_as_image
from tqdm import tqdm

!pip install torchgeo
import torchgeo.models

from datetime import datetime

from tifffile import imread

from sklearn.metrics import confusion_matrix
from sklearn import model_selection

import torch, torchvision
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import transforms
from torchvision.transforms import ToTensor

     |████████████████████████████████| 19.3 MB 5.2 MB/s 
     |████████████████████████████████| 204 kB 5.1 MB/s 
     |████████████████████████████████| 6.3 MB 59.4 MB/s 
     |████████████████████████████████| 88 kB 7.8 MB/s 
     |████████████████████████████████| 16.7 MB 66.0 MB/s 
     |████████████████████████████████| 78 kB 8.9 MB/s 
     |████████████████████████████████| 584 kB 72.1 MB/s 
     |████████████████████████████████| 512 kB 71.0 MB/s 
     |████████████████████████████████| 1.0 MB 73.0 MB/s 
     |████████████████████████████████| 431 kB 79.8 MB/s 
     |████████████████████████████████| 409 kB 89.8 MB/s 
     |████████████████████████████████| 117 kB 88.2 MB/s 
     |████████████████████████████████| 596 kB 59.8 MB/s 
     |████████████████████████████████| 140 kB 71.3 MB/s 
     |████████████████████████████████| 1.1 MB 59.3 MB/s 
     |████████████████████████████████| 376 kB 66.5 MB/s 
     |████████████████████████████████| 58 kB 4.9 MB/s 
     |██████████████

In [3]:
#local
#Loads tiff folders

import os
folder = '/content/drive/MyDrive/ML/data/dataset/trainset/'
#folder = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset'
subfolders = []
for dirs in os.walk(folder):
  subfolders.append(dirs[0])

#remove directory
subfolders.pop(0)
for i in range(0, len(subfolders)):
     print(subfolders[i])

/content/drive/MyDrive/ML/data/dataset/trainset/AnnualCrop
/content/drive/MyDrive/ML/data/dataset/trainset/Forest
/content/drive/MyDrive/ML/data/dataset/trainset/HerbaceousVegetation
/content/drive/MyDrive/ML/data/dataset/trainset/Highway
/content/drive/MyDrive/ML/data/dataset/trainset/Industrial
/content/drive/MyDrive/ML/data/dataset/trainset/Pasture
/content/drive/MyDrive/ML/data/dataset/trainset/PermanentCrop
/content/drive/MyDrive/ML/data/dataset/trainset/Residential
/content/drive/MyDrive/ML/data/dataset/trainset/River
/content/drive/MyDrive/ML/data/dataset/trainset/SeaLake


In [4]:
#Define paths

ROOT_PATH = '/content/drive/MyDrive/ML/'
#ROOT_PATH = '/Volumes/GoogleDrive/My Drive/ML'
BASE_PATH = os.path.join(ROOT_PATH, 'data')
MODEL_PATH = os.path.join(ROOT_PATH, 'model')

file_path_train = '/content/drive/MyDrive/ML/data/dataset/train.csv'
file_path_test = '/content/drive/MyDrive/ML/data/dataset/test.csv'
#file_path_train = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/train.csv'
#file_path_test = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/test.csv'
data_df_train = pd.read_csv(file_path_train)
data_df_test = pd.read_csv(file_path_test)

data_df_train.columns = data_df_train.columns.str.replace('path', 'image_id')
data_df_train.columns = data_df_train.columns.str.replace('Label', 'label')

In [5]:
data_df_train.head()

,image_id,label
0,AnnualCrop/AnnualCrop_135.tif,AnnualCrop
1,AnnualCrop/AnnualCrop_692.tif,AnnualCrop
2,AnnualCrop/AnnualCrop_2218.tif,AnnualCrop
3,AnnualCrop/AnnualCrop_2679.tif,AnnualCrop
4,AnnualCrop/AnnualCrop_926.tif,AnnualCrop


In [6]:
type(data_df_train)

pandas.core.frame.DataFrame

In [7]:
#Init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

In [8]:
#Classes and labels

IDX_CLASS_LABELS = {
    0: 'AnnualCrop',
    1: 'Forest', 
    2: 'HerbaceousVegetation',
    3: 'Highway',
    4: 'Industrial',
    5: 'Pasture',
    6: 'PermanentCrop',
    7: 'Residential',
    8: 'River',
    9: 'SeaLake'
}

CLASSES = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture','PermanentCrop','Residential','River', 'SeaLake']
CLASS_IDX_LABELS = dict()
for key, val in IDX_CLASS_LABELS.items():
  CLASS_IDX_LABELS[val] = key

NUM_CLASSES = len(IDX_CLASS_LABELS.items())
torch.manual_seed(10)
VALID_SIZE = 0.1

In [9]:
#Encoder and decoder

## Give idx of each class name
def encode_label(label):
    idx = CLASS_IDX_LABELS[label] 
    return idx

## Take in idx and return the class name
def decode_target(target, text_labels=True):
    result = []
    if text_labels:
        return IDX_CLASS_LABELS[target]
    else:
        return target

In [10]:
#Open tiff files with rasterio and safe to dictionary

'''import os

data = {}

for i in range(len(subfolders)):

  txtfiles = []

  # traverse whole directory
  for root, dirs, files in os.walk(subfolders[i]):
    # select file name
    for file in files:
        # check the extension of files
        if file.endswith('.tif'):
            txtfiles.append(os.path.join(root, file))

  code = subfolders[i].split('/')[8]
  index = encode_label(code)
  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")

  print(code)
  print(index)

  i = 0
  for pic in txtfiles:
    with rasterio.open(pic, "r") as img:
      arr[i] = np.moveaxis(img.read(), 0, 2)
      i += 1
      
  data[index] = arr'''

'import os\n\ndata = {}\n\nfor i in range(len(subfolders)):\n\n  txtfiles = []\n\n  # traverse whole directory\n  for root, dirs, files in os.walk(subfolders[i]):\n    # select file name\n    for file in files:\n        # check the extension of files\n        if file.endswith(\'.tif\'):\n            txtfiles.append(os.path.join(root, file))\n\n  code = subfolders[i].split(\'/\')[8]\n  index = encode_label(code)\n  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")\n\n  print(code)\n  print(index)\n\n  i = 0\n  for pic in txtfiles:\n    with rasterio.open(pic, "r") as img:\n      arr[i] = np.moveaxis(img.read(), 0, 2)\n      i += 1\n      \n  data[index] = arr'

In [11]:
#Save the files as .npy

'''for i in range(len(data)):
  keys_list = list(data)
  key = keys_list[i]
  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)'''

"for i in range(len(data)):\n  keys_list = list(data)\n  key = keys_list[i]\n  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)"

In [12]:
#Reload the .npy files

import os
path = '/content/drive/MyDrive/ML/data/dataset/trainset/'
#path = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset'
npyfiles = []
data = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in tqdm(range(len(npyfiles))):
  path = npyfiles[i]
  arr = np.load(path)
  index = path.split('/')[8].split('_')[1][5]
  index = int(index)
  data[index] = arr

100%|██████████| 10/10 [00:32<00:00,  3.22s/it]


In [13]:
from pathlib import Path
class EuroSAT(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        row = self.train_df.loc[idx]
        img_id, label = row['image_id'], row['label']
        img_id = int(img_id.split('.')[0].split('_')[1])
        label_encode = int(encode_label(label))
        id = img_id - 1
        img = data[label_encode][id]
        img_rgb = img[:, :, [3,2,1]]
        if self.transform:
            img_rgb = self.transform(img_rgb)
        return img_rgb, encode_label(label)

In [14]:
transform_normalisation = transforms.Compose([
    transforms.ToTensor(),           
    transforms.ConvertImageDtype(torch.float),
])

In [15]:
train_ds = EuroSAT(data_df_train, BASE_PATH, transform_normalisation)
print(len(train_ds))

27000


In [16]:
normalisation_dl = DataLoader(train_ds, batch_size=len(train_ds))

In [17]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [18]:
mean, std = get_mean_and_std(normalisation_dl)

In [19]:
print(mean)
print(std)

tensor([ 946.5516, 1041.8866, 1117.2065])
tensor([594.4782, 395.2131, 333.4205])


In [174]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Resize(224),              
    transforms.ConvertImageDtype(torch.float),
    #transforms.RandomHorizontalFlip(p=0.35),
    #transforms.RandomVerticalFlip(p=0.5),
    #transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    #transforms.RandomRotation(degrees=(30, 70)),
    transforms.Normalize(
        mean=[946.5544, 1041.8865, 1117.2018],
        std=[594.4802, 395.2151, 333.4477]
    )
])

transformer_train = transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Resize(224),
                                  transforms.RandomRotation(5),
                                  transforms.RandomHorizontalFlip(0.5),
                                  transforms.RandomCrop(224, padding = 10),
                                  transforms.Normalize(
        mean=[946.5544, 1041.8865, 1117.2018],
        std=[594.4802, 395.2151, 333.4477]
    )
])

In [176]:
train_ds = EuroSAT(data_df_train, BASE_PATH, transformer_train)
print(len(train_ds))

27000


In [177]:
train_size = int(0.8 * len(data_df_train))
test_size = len(data_df_train) - train_size
train_ds, validation_ds = torch.utils.data.random_split(train_ds, [train_size, test_size], generator=torch.Generator().manual_seed(seed_value))

In [ ]:
train_ds[0]

In [24]:
print(len(train_ds))
print(len(validation_ds))

21600
5400


In [179]:
model = models.resnet50(pretrained=True)

Model

In [ ]:
model

In [192]:
pretrainedBool = True
if pretrainedBool:
    for param in model.parameters():
        param.requires_grad = False
model.fc = nn.Sequential(
                      nn.Linear(2048, 256),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(256, NUM_CLASSES),
                      nn.LogSoftmax(dim=1)
                        )

In [193]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
print('[LOG] notebook with {} computation enabled'.format(str(device)))

[LOG] notebook with cuda computation enabled


In [194]:
model = model.to(device)

In [195]:
# define the optimization criterion / loss function
ce_loss = nn.CrossEntropyLoss()

In [196]:
ce_loss = ce_loss.to(device)

In [197]:
config={
"epochs": 20,
"mini_batch_size": 64,
"learning_rate": 0.001,
"opt_func": torch.optim.Adam,
"milestones": [5,12],
"weight_decay": 0,
"description": "resnet50_with_nor_and_224"
}

In [198]:
train_dl = DataLoader(train_ds, batch_size=config.get("mini_batch_size"), shuffle=True)

In [199]:
#Empty cache
torch.cuda.empty_cache()

In [35]:
!pip install wandb
!wandb login
import wandb
#Key: 9da448bfaa162b572403e1551114a17058f249d0

     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 145 kB 88.8 MB/s 
     |████████████████████████████████| 181 kB 68.0 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=7381748d29b069e9638cc7259eea2b781caa633bdf3936d14cdae3af45a0deb0
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [189]:
run = wandb.init(project="ml-challenge", entity="nicikess", config=config)

loss,█▅▅▄▃▃▂▃▂▂▂▂▃▃▂▂▁▂▂▁▁▂▃▂▂▃▂▂▁▂▁▁▁▂▁▁▁▂▁▃
loss,0.28739


In [200]:
def train(model,train_dl,epochs,learning_rate,opt_func,milestones,weight_decay):
  
  # init collection of training epoch losses
  train_epoch_losses = []

  # set the model in training mode
  model.train()

  optimizer = opt_func(model.fc.parameters(),lr=learning_rate, weight_decay=weight_decay)
  scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1)

  # train the CIFAR10 model
  for epoch in range(epochs):
      
      # init collection of mini-batch losses
      train_mini_batch_losses = []

      # iterate over all-mini batches
      for i, (images, labels) in enumerate(train_dl):

          #print(images)

          # push mini-batch data to computation device
          images = images.to(device)
          labels = labels.to(device)

          # run forward pass through the network
          output = model(images)

          # reset graph gradients
          optimizer.zero_grad()
          
          # determine classification loss
          loss = ce_loss(output, labels)

          wandb.log({"loss": loss})
          
          # run backward pass
          loss.backward()
          
          # update network paramaters
          optimizer.step()
          
          # collect mini-batch reconstruction loss
          train_mini_batch_losses.append(loss.data.item())

      # determine mean min-batch loss of epoch
      train_epoch_loss = np.mean(train_mini_batch_losses)

      print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))

      scheduler.step()

      print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))

      #Added
      #result = evaluate(model, vali_dataloader)

      # print epoch loss
      now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
      print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))
      
      #if (epoch%10 == 0): 
          # set filename of actual model
      model_name = 'challenge_model_epoch_{}.pth'.format(str(epoch))
          # save current model to GDrive models directory
      torch.save(model.state_dict(), os.path.join(MODEL_PATH, model_name))

      # determine mean min-batch loss of epoch
      train_epoch_losses.append(train_epoch_loss)

In [201]:
epochs = config.get("epochs")
learning_rate = config.get("learning_rate")
weight_decay = config.get("weight_decay")
opt_func = config.get("opt_func")
milestones = config.get("milestones")
max_learning_rate = config.get("max_learning_rate")

train(model,train_dl,epochs,learning_rate,opt_func,milestones,weight_decay)

Epoch-0 lr: 0.001
Epoch-0 lr: 0.001
[LOG 20220522-09:54:26] epoch: 0 train-loss: 0.5321311803876295
Epoch-1 lr: 0.001
Epoch-1 lr: 0.001
[LOG 20220522-09:56:40] epoch: 1 train-loss: 0.32818379261966285
Epoch-2 lr: 0.001
Epoch-2 lr: 0.001
[LOG 20220522-09:58:54] epoch: 2 train-loss: 0.2923617624738336
Epoch-3 lr: 0.001
Epoch-3 lr: 0.001
[LOG 20220522-10:01:09] epoch: 3 train-loss: 0.27416270038873486
Epoch-4 lr: 0.001
Epoch-4 lr: 0.0001
[LOG 20220522-10:03:23] epoch: 4 train-loss: 0.2589891870125864
Epoch-5 lr: 0.0001
Epoch-5 lr: 0.0001
[LOG 20220522-10:05:37] epoch: 5 train-loss: 0.20037232841965716
Epoch-6 lr: 0.0001
Epoch-6 lr: 0.0001
[LOG 20220522-10:07:51] epoch: 6 train-loss: 0.19938017683621695
Epoch-7 lr: 0.0001
Epoch-7 lr: 0.0001
[LOG 20220522-10:10:06] epoch: 7 train-loss: 0.19292270628041064
Epoch-8 lr: 0.0001
Epoch-8 lr: 0.0001
[LOG 20220522-10:12:21] epoch: 8 train-loss: 0.18647092843108629
Epoch-9 lr: 0.0001
Epoch-9 lr: 0.0001
[LOG 20220522-10:14:35] epoch: 9 train-loss: 0.

In [ ]:
resnet50_vali_dataloader = DataLoader(validation_ds, batch_size=5400)

In [ ]:
# Display image and label.
validation_features, validation_labels = next(iter(resnet50_vali_dataloader))
print(f"Labels batch shape: {validation_labels.size()}")

In [ ]:
# restore pre-trained model snapshot
best_model_name = os.path.join(MODEL_PATH, 'challenge_model_epoch_12.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = model

# load pre-trained models
best_model.load_state_dict(state_dict_best)

In [ ]:
# set model in evaluation mode
best_model.eval()

In [ ]:
predictionsVal = list()
from PIL import Image
from numpy import vstack
for i in tqdm(range(len(validation_ds))):
  input = validation_ds[i]
  input = input[0].unsqueeze(0)
  #print(input)
  #input = test_transform(input)
  #x = x[None, :]
  input = input.to(device)
  #print(type(input))
  yhat = best_model(input)
  # retrieve numpy array
  yhat = yhat.detach()
  # convert to class labels
  yhat = torch.argmax(yhat, axis=1)
  # reshape for stacking
  yhat = yhat.reshape((len(yhat), 1))
  # store
  predictionsVal.append(yhat.cpu())
predictionsVal = vstack(predictionsVal)
predictionsVal = torch.tensor(predictionsVal)

In [ ]:
np.shape(validation_ds[3][0])

In [ ]:
list_tens = list()

for i in range(len(predictionsVal)):
  list_tens.append(decode_target(predictionsVal[i].item()))

In [ ]:
from collections import Counter
print(Counter(list_tens).keys()) # equals to list(set(words))
print(Counter(list_tens).values()) # counts the elements' frequency

In [ ]:
from sklearn import metrics
acc = metrics.accuracy_score(validation_labels.detach().cpu(), predictionsVal.detach().cpu())
print(acc)

In [ ]:
wandb.log({"acc": acc})

In [39]:
#Finish run
run.finish()

loss,█▇▅█▄▇▂▂▃▃▄▃▁▃▄▂▂▂▄▄▄▁▃▂▂▅▃▂▃▃▂▂▃▃▄▃▂▁▃▃
loss,0.1743


In [40]:
import os
path = '/content/drive/MyDrive/ML/data/dataset/testset/'
npyfiles = []
datatest = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in tqdm(range(len(npyfiles))):
  path = npyfiles[i]
  arr = np.load(path)
  index = int(path.split('_')[1].split('.')[0])
  datatest[index] = arr

100%|██████████| 4232/4232 [00:57<00:00, 73.69it/s] 


In [41]:
from pathlib import Path
class EuroSATTest(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    #def __getitem__(self, idx):
        img = datatest[idx]
        img = img[:, :, [4,3,2]]
        img = np.float32(img)
        if self.transform:
            img = self.transform(img)
        return img

In [42]:
def get_mean_and_std_test(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [43]:
test_ds = EuroSATTest(data_df_test, BASE_PATH, transform_normalisation)
normalisation_dl = DataLoader(test_ds, batch_size=len(test_ds))
mean, std = get_mean_and_std_test(normalisation_dl)

In [ ]:
print(std)

In [57]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Resize(224),              
    transforms.ConvertImageDtype(torch.float),
    #transforms.RandomHorizontalFlip(p=0.5),
    #transforms.RandomVerticalFlip(p=0.5),
    #transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    #transforms.RandomRotation(degrees=(30, 70)),
    transforms.Normalize(
        mean=[943.4290, 578.8717, 628.8640],
        std=[660.1735, 599.4288, 486.5515]
    )
])

In [58]:
#Test data
test_ds = EuroSATTest(data_df_test, BASE_PATH, test_transform)
print(len(test_ds))

4232


In [ ]:
test_ds[0]

In [139]:
# restore pre-trained model snapshot
best_model_name = os.path.join(MODEL_PATH, 'challenge_model_epoch_9.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = model

# load pre-trained models
best_model.load_state_dict(state_dict_best)

<All keys matched successfully>

In [ ]:
# set model in evaluation mode
best_model.eval()

In [141]:
predictions = list()
from PIL import Image
from numpy import vstack
for i in tqdm(range(len(test_ds))):
  input = test_ds[i]
  input = input.unsqueeze(0)
  #print(input)
  #input = test_transform(input)
  #x = x[None, :]
  input = input.to(device)
  #print(type(input))
  yhat = best_model(input)
  # retrieve numpy array
  yhat = yhat.detach()
  # convert to class labels
  yhat = torch.argmax(yhat, axis=1)
  # reshape for stacking
  yhat = yhat.reshape((len(yhat), 1))
  # store
  predictions.append(yhat.cpu())
predictions = vstack(predictions)
predictions = torch.tensor(predictions)

100%|██████████| 4232/4232 [00:28<00:00, 147.59it/s]


In [142]:
list_tens = list()

for i in range(len(predictions)):
  list_tens.append(decode_target(predictions[i].item()))

In [143]:
from collections import Counter
print(Counter(list_tens).keys()) # equals to list(set(words))
print(Counter(list_tens).values()) # counts the elements' frequency

dict_keys(['PermanentCrop', 'Highway', 'Residential', 'SeaLake', 'Forest', 'AnnualCrop', 'HerbaceousVegetation', 'Industrial', 'River', 'Pasture'])
dict_values([1010, 443, 711, 930, 241, 442, 80, 24, 86, 265])


In [ ]:
from sklearn import metrics
metrics.accuracy_score(validation_labels.detach().cpu(), predictions.detach().cpu())

In [76]:
test_indices_path = '/content/drive/MyDrive/ML/data/dataset/test.csv'

final_submission_file = pd.read_csv(test_indices_path)
prediction_list = []

# turning numeric predictions to strings (as in kaggle example submission)
for p in predictions:
  prediction_list.append(decode_target(p.item()))

final_submission_file["label"] = prediction_list
### source with alternatives methods for appending to csv should this fail: https://stackoverflow.com/questions/26666919/add-column-in-dataframe-from-list
final_submission_file.to_csv('/content/drive/MyDrive/ML/data/dataset/final_submission.csv', index=None)